In [11]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import folium

In [12]:
df = pd.read_csv("400cities_20241029.csv")

In [13]:
# Convert the 'geometry' column from WKT to shapely geometries
df['geometry'] = df['geometry'].apply(wkt.loads)

In [14]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:3857')
gdf = gdf.to_crs(epsg=4326) # Transform to WGS84 (EPSG:4326)

In [15]:
# Extract longitude and latitude from the transformed geometries
gdf['lon'] = gdf.geometry.x
gdf['lat'] = gdf.geometry.y

In [16]:
gdf['label'] = gdf['CITY_NAME']+", "+gdf['CNTRY_NAME']+": "+gdf['city_type']

In [17]:
gdf[gdf.capital==1]

,FID,CITY_NAME,GMI_ADMIN,ADMIN_NAME,FIPS_CNTRY,CNTRY_NAME,STATUS,POP,POP_RANK,POP_CLASS,...,TYPE__sea,ISO_CC__sea,SHAPE_Length__sea,SHAPE_Area__sea,distance_sea,city_type,pop_diff,lon,lat,label
0,296,Oranjestad,ABW,Aruba,AA,Aruba,National and provincial capital,28295,7,"Less than 50,000",...,Ocean or Sea,NaN,1.283472,0.053652,1.449000,coast,NaN,-70.026003,12.525003,"Oranjestad, Aruba: coast"
1,1556,Saint John's,ATG,Antigua & Barbuda,AC,Antigua & Barbuda,National capital,0,0,,...,Ocean or Sea,NaN,1.896968,0.045993,0.684450,coast,NaN,-61.842177,17.124777,"Saint John's, Antigua & Barbuda: coast"
2,2257,Abu Dhabi,ARE-AZA,Abu Zaby,AE,United Arab Emirates,National and provincial capital,1539000,2,"1,000,000 to 4,999,999",...,Ocean or Sea,NaN,3.565310,0.047519,2.347458,coast,NaN,54.370998,24.476004,"Abu Dhabi, United Arab Emirates: coast"
3,1072,Kabul,AFG-KAB,Kabol,AF,Afghanistan,National and provincial capital,4136000,2,"1,000,000 to 4,999,999",...,Ocean or Sea,NaN,0.702247,0.009304,1192.107975,inland,NaN,69.136758,34.530912,"Kabul, Afghanistan: inland"
4,1910,Algiers,DZA-ALG,Alger,AG,Algeria,National and provincial capital,3608000,2,"1,000,000 to 4,999,999",...,Ocean or Sea,NaN,1.359198,0.047538,1.553963,coast,NaN,3.050001,36.783297,"Algiers, Algeria: coast"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,17,Apia,WSM,Samoa,WS,Samoa,National capital,37391,7,"Less than 50,000",...,Ocean or Sea,NaN,1.146609,0.045098,0.000000,coast,NaN,-171.739494,-13.837023,"Apia, Samoa: coast"
285,2458,Mbabane,SWZ-HHH,Hhohho,WZ,Eswatini,National capital,0,0,,...,Ocean or Sea,NaN,0.553750,0.010466,157.163254,inland,NaN,31.191300,-26.303382,"Mbabane, Eswatini: inland"
286,865,Sanaa,YEM-SAN,San'a',YM,Yemen,National and provincial capital,3820000,2,"1,000,000 to 4,999,999",...,Ocean or Sea,NaN,1.752070,0.069197,149.616375,inland,NaN,44.209503,15.361443,"Sanaa, Yemen: inland"
288,2382,Lusaka,ZMB-LUS,Lusaka,ZA,Zambia,National and provincial capital,1747152,2,"1,000,000 to 4,999,999",...,Ocean or Sea,NaN,0.521008,0.008197,897.500478,inland,NaN,28.170000,-15.429996,"Lusaka, Zambia: inland"


In [18]:
city_map = folium.Map(zoom_start=6, min_zoom=2)
for i in range(len(gdf)):
    if gdf.capital.iloc[i]==1: # capital
        folium.Marker(location=[gdf.lat.iloc[i], gdf.lon.iloc[i]], popup=gdf.label.iloc[i], icon=folium.Icon(color="darkred", icon="flag")).add_to(city_map)
    else: # non capital
        folium.Marker(location=[gdf.lat.iloc[i], gdf.lon.iloc[i]], popup=gdf.label.iloc[i], icon=folium.Icon(color="darkgreen", icon="flag")).add_to(city_map)

In [19]:
city_map